In [1]:
import netgen.geom2d as geom2d
from ngsolve import *
from ngsolve.webgui import Draw


geo = geom2d.SplineGeometry()
pnums = [ geo.AddPoint (x,y,maxh=0.01) for x,y in [(0,0), (1,0), (1,0.1), (0,0.1)] ]
for p1,p2,bc in [(0,1,"bot"), (1,2,"right"), (2,3,"top"), (3,0,"left")]:
     geo.Append(["line", pnums[p1], pnums[p2]], bc=bc)
mesh = Mesh(geo.GenerateMesh(maxh=0.05))

In [35]:
#

# geometric non-linear elasticity with Neo-Hooke hyperelastic material
#
# featuring automatic differentiation in SymbolicEnergy
#

E, nu = 210, 0.2
mu  = E / 2 / (1+nu)
lam = E * nu / ((1+nu)*(1-2*nu))

fes = H1(mesh, order=2, dirichlet="left", dim=mesh.dim)
# fes = VectorH1(mesh, order=2, dirichlet="left")

u  = fes.TrialFunction()

force = CoefficientFunction( (0,1/2) )

I = Id(mesh.dim)
F = I + Grad(u)
C = F.trans * F
E = 0.5 * (C-I)

def Pow(a, b):
    return a**b  # exp (log(a)*b)
  
def NeoHooke (C):
    return 0.5 * mu * (Trace(C-I) + 2*mu/lam * Pow(Det(C),-lam/2/mu) - 1)



factor = Parameter(1)

a = BilinearForm(fes, symmetric=False)
a += Variation (NeoHooke(C).Compile()*dx)
a += Variation ((-factor * InnerProduct(force,u) ).Compile()*dx)


u = GridFunction(fes)
u.vec[:] = 0

res = u.vec.CreateVector()
w = u.vec.CreateVector()


for loadstep in range(10):
    
    print ("loadstep", loadstep)
    factor.Set (loadstep+0.5)
    
    for it in range(5):
        print ("Newton iteration", it)
        print()
        print ("energy = ", a.Energy(u.vec))
        a.Apply(u.vec, res)
        a.AssembleLinearization(u.vec)
        inv = a.mat.Inverse(fes.FreeDofs() ) 
        w.data = inv*res
        print ("err^2 = ", InnerProduct (w,res))
        u.vec.data -= w


loadstep 0
Newton iteration 0

energy =  8.750000000000005
err^2 =  0.001738552744825331
Newton iteration 1

energy =  8.759390470627046
err^2 =  0.020860660425452312
Newton iteration 2

energy =  8.749147002798868
err^2 =  1.4997079029489234e-05
Newton iteration 3

energy =  8.749139511967497
err^2 =  1.7239980736551602e-08
Newton iteration 4

energy =  8.74913950334953
err^2 =  3.0781270061658627e-12
loadstep 1
Newton iteration 0

energy =  8.745731742697535
err^2 =  0.006548583505978049
Newton iteration 1

energy =  8.864911109481403
err^2 =  0.2585030561409747
Newton iteration 2

energy =  8.74385899218876
err^2 =  0.0021372227645131132
Newton iteration 3

energy =  8.74281420034671
err^2 =  7.914253740211221e-05
Newton iteration 4

energy =  8.742783008083148
err^2 =  1.8250970163182507e-05
loadstep 2
Newton iteration 0

energy =  8.733795227025164
err^2 =  0.004494152832020552
Newton iteration 1

energy =  8.769690294222368
err^2 =  0.07802467277932155
Newton iteration 2

energy 

In [4]:

E, nu = 210, 0.2
mu  = E / 2 / (1+nu)
lam = E * nu / ((1+nu)*(1-2*nu))

Pspace = NumberSpace(mesh, order=2, dirichlet="left") 
Uspace = VectorH1(mesh,order=2, dirichlet="left") 
Pispace = MatrixValued(L2(mesh,order = 2, dirichlet = 'left'), symmetric=True)
 

fes = FESpace([Pispace, Uspace, Pspace])
PI,U,P  = fes.TrialFunction()

TAU, V, Q = fes.TestFunction()

force = CoefficientFunction( (0,1/2) )

I = Id(mesh.dim)
F = I + Grad(U)
C = F.trans * F
E = 0.5 * (C-I)

def Pow(a, b):
    return a**b  # exp (log(a)*b)
  
def Neo_Dev(F):
    return 0.5 * mu * (F + F.trans - (Det(F.trans * F)**(-lam/2/mu)) * Cof(F.trans * F).trans * (F + F.trans))



factor = Parameter(1)

a = BilinearForm(fes)
a += InnerProduct(PI - Neo_Dev(F)*(Id(mesh.dim) + Grad(U)).trans + P * Id(mesh.dim), TAU).Compile()*dx
a += InnerProduct(PI*Inv(Id(mesh.dim) + Grad(U)).trans, Grad(V)).Compile() * dx
a += ((Det(Grad(U)+ Id(mesh.dim)) - 1) * Q).Compile() * dx

a += Variation(-factor * InnerProduct(force,U) .Compile()*dx)


u = GridFunction(fes)
u.vec[:] = 0

res = u.vec.CreateVector()
w = u.vec.CreateVector()


for loadstep in range(10):
    
    print ("loadstep", loadstep)
    factor.Set (loadstep+1)
    
    for it in range(5):
        print ("Newton iteration", it)
        print ("energy = ", a.Energy(u.vec))
        a.Apply(u.vec, res)
        a.AssembleLinearization(u.vec)
        inv = a.mat.Inverse(fes.FreeDofs() ) 
        w.data = inv*res
        print ("err^2 = ", InnerProduct (w,res))
        u.vec.data -= w




loadstep 0
Newton iteration 0
energy =  0.0
err^2 =  0.10635999221044132
Newton iteration 1
energy =  -763.0962942145628
err^2 =  -32780136.800760597
Newton iteration 2
energy =  -59398447764.69258
err^2 =  430684689709407.5
Newton iteration 3
energy =  -58305497782075.836
err^2 =  -117675859052956.34
Newton iteration 4
energy =  27167002666214.94
err^2 =  121627129045049.48
loadstep 1
Newton iteration 0
energy =  -326642809980344.9
err^2 =  -1.0125783629527476e+16
Newton iteration 1
energy =  1.528005547790958e+16
err^2 =  -2.4575516729116438e+17
Newton iteration 2
energy =  -1.689626412139033e+19
err^2 =  6.32378309609429e+21
Newton iteration 3
energy =  5.112685733384932e+20
err^2 =  -4.8120157363904834e+22
Newton iteration 4
energy =  2.824172126116144e+24
err^2 =  -9.030737796260315e+26
loadstep 2
Newton iteration 0
energy =  1.1002157890179343e+26
err^2 =  1.9632796846332013e+27
Newton iteration 1
energy =  -1.3203708180802226e+26
err^2 =  -1.530652558733762e+27
Newton iteration 

In [8]:
Draw (u.components[1]*1e-15, mesh, "disp")

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene